In [3]:
# import names
# import syft as sy
import numpy as np
# from syft.lib.autograd.value import Value, to_values, grad
# from syft.lib.adp.entity import Entity
# from syft.lib.adp.scalar import Scalar
# from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.tensor import Tensor

In [4]:
x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1)
y = Tensor(np.array([[0],[0],[1],[1]])).private(min_val=0, max_val=1)

In [5]:
z = y + 1

True


In [446]:
z.is_private

False

In [331]:
weights = PrivateTensor(np.random.uniform(size=(2,1)))

In [332]:
pred = x.dot(weights)
loss = np.mean(np.square(y-pred))
loss.backward()

weights.grad.value

array([[1.16290282],
       [0.49474294]])

In [307]:
weights.value

AttributeError: 'numpy.ndarray' object has no attribute 'value'

In [300]:
weights.value

array([[0.43256615],
       [0.25068221]])

In [301]:
loss.value

array(0.59035326)

In [117]:
for i in range(1):
    pred = x.dot(weights)
    loss = np.mean(np.square(y-pred))
    grads = grad(loss)

#     for w in weights.squeeze().tolist():
#         w.value -= w._grad.value * 0.1

#     print(loss.value.value)

TypeError: unhashable type: 'PrivateTensor'

In [80]:
# {weight:grads[weight.id].value for weight in weights.squeeze().tolist()}

In [12]:
v._grad.value

0.3663145140234199

In [7]:
weights1 = to_values(np.random.uniform(size=(1,12))) #1 feature -> 12 features
weights2 = to_values(np.random.uniform(size=(12,1))) #12 features -> 2 predictions

In [8]:
layer1 = x@weights1
nonlin1 = sigmoid(layer1)
y_pred = nonlin1@weights2
loss = np.mean(np.square(y-y_pred))

In [9]:
%%time
grads = grad(loss)
gradients = {item:grads[item] for item in ("i", "j", "k", "l", "m", "n", "o", "p")}

CPU times: user 5.77 s, sys: 22.5 ms, total: 5.79 s
Wall time: 5.81 s


In [10]:
i_grads = gradients["i"]
i_grads

0.000917289234263041*(-0.089751527921597*q + 0.0644426941140896/(1 + exp(-0.000458644617131521*i)) + 0.0535038305420718/(1 + exp(-0.11511227410626*i)) + 0.0264646714999528/(1 + exp(-0.120036767489687*i)) + 0.0466940644125706/(1 + exp(-0.335310888711682*i)) + 0.0566148092339863/(1 + exp(-0.352757926684868*i)) + 0.0256511451704527/(1 + exp(-0.353901978140667*i)) + 0.016497246454841/(1 + exp(-0.480682924441222*i)) + 0.0405016539183177/(1 + exp(-0.538365557430182*i)) + 0.0592537002226106/(1 + exp(-0.649192077027798*i)) + 0.0766515615055929/(1 + exp(-0.743328555064979*i)) + 0.0445657625775947/(1 + exp(-0.86918849570602*i)) + 0.0750109013333917/(1 + exp(-0.910735853618278*i)))*exp(-0.000458644617131521*i)/(1 + exp(-0.000458644617131521*i))**2 + 0.23022454821252*(-0.0745166012505247*q + 0.0535038305420718/(1 + exp(-0.000458644617131521*i)) + 0.0444217908954375/(1 + exp(-0.11511227410626*i)) + 0.0219724100419862/(1 + exp(-0.120036767489687*i)) + 0.0387679525816804/(1 + exp(-0.335310888711682*i

In [11]:
i_grads.free_symbols

{i, q}

In [12]:
to_optimize = -i_grads #minimize the negative

In [13]:
bounds = [tuple([np.random.uniform(1, 11),np.random.uniform(11,20)]) for _ in range(len(i_grads.free_symbols))]

In [14]:
bounds

[(10.579387090860122, 11.52316946554759),
 (5.872435993196004, 11.020894248689707)]

In [15]:
func = sy.lambdify(list(i_grads.free_symbols), to_optimize)

In [16]:
def func_for_scipy(x):
    return func(*tuple(x))

In [17]:
sol = shgo(func_for_scipy, bounds=bounds)
sol

     fun: 0.03458835150139999
    funl: array([0.03458835])
 message: 'Optimization terminated successfully.'
    nfev: 8
     nit: 2
   nlfev: 3
   nlhev: 0
   nljev: 1
 success: True
       x: array([10.57938709, 11.02089425])
      xl: array([[10.57938709, 11.02089425]])

In [18]:
sol.x

array([10.57938709, 11.02089425])

In [ ]:
# alice = Scalar(value=0.1, min_val=-1, max_val=1, entity=Entity(name="Alice"))
# bob = Scalar(value=0.01, min_val=-2, max_val=2, entity=Entity(name="Bob"))
# charlie = Scalar(value=0.001, min_val=-2, max_val=2, entity=Entity(name="Charlie"))
# david = Scalar(value=0.0001, min_val=-2, max_val=2, entity=Entity(name="David"))